In [62]:
import dropbox
import urllib.request, json 
from collections import defaultdict
from collections import OrderedDict
import datetime as DT
import ast
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np
from string import punctuation
import csv
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

# Helper libraries
from w266_common import utils, vocabulary

########################### DLEE IMPORTS ##################################

# Import pandas and tensorflow
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1.10"))

# Import model
import models

# Import misc
from importlib import reload
import os, sys, re, json, time, datetime, shutil
import itertools
from w266_common import patched_numpy_io

In [80]:
##Download the pre-processed tweets from dropbox

access_token = 'p3Ze8FpdRTAAAAAAAAAADb9hCQ8aXXYU3A3gGM1HEXyeMrs8KI2SrA71KDmmCXte'
dbx = dropbox.Dropbox(access_token)
metadata, res = dbx.files_download(path="/Inputdata2.txt")


access_token2 = 'm_fKi8mWZ0AAAAAAAAAADlfzTm37f1y45k92Xpiw1B6mjT3dQqzNvKgpIUSR17uq'
dbx2 = dropbox.Dropbox(access_token2)

#Write the combined file into the drive
with open("InputX.txt", "wb") as f:
    metadata, res = dbx.files_download(path="/Inputdata2.txt")
    metadata2, res2 = dbx2.files_download(path="/Inputdata.txt")
    f.write(res.content)
    f.write(res2.content)

In [63]:
##Translate date of the tweet to quarter reference
def Quarterlookup(date):
    if date>=DT.datetime(2018,10,1):
        return '18Q4'
    elif date>=DT.datetime(2018,7,1):
        return '18Q3'
    elif date>=DT.datetime(2018,4,1):
        return '18Q2'
    elif date>=DT.datetime(2018,1,1):
        return '18Q1'
    elif date>=DT.datetime(2017,10,1):
        return '17Q4'
    elif date>=DT.datetime(2017,7,1):
        return '17Q3'
    elif date>=DT.datetime(2017,4,1):
        return '17Q2'
    elif date>=DT.datetime(2017,1,1):
        return '17Q1'
    elif date>=DT.datetime(2016,10,1):
        return '16Q4'
    elif date>=DT.datetime(2016,7,1):
        return '16Q3'
    elif date>=DT.datetime(2016,4,1):
        return '16Q2'
    elif date>=DT.datetime(2016,1,1):
        return '16Q1'
    elif date>=DT.datetime(2015,10,1):
        return '15Q4'
    elif date>=DT.datetime(2015,7,1):
        return '15Q3'
    elif date>=DT.datetime(2015,4,1):
        return '15Q2'
    else:
        return '15Q1'

In [64]:
##Reading the Input X file and process it
d=defaultdict(list)

with open('InputX.txt') as f:
    for line in f:
        count=0
        temp =-1
        k,v=line.split(':',1)
        while True:
            
            oldtemp=temp
            temp=v.find(']]',temp+1)
            if count==0:
                d[k].append(v[oldtemp+2:temp+2]) 
            else:
                d[k].append(v[oldtemp+4:temp+2])
           
             
            count+=1
            if temp == -1: break
#             if count == 500: break  # initially limit for time sake
    

In [65]:
#Read from the vocab file pre prepared
with open('vocab.csv', 'r') as f:
    reader = csv.reader(f,delimiter=';')
    allword = set(list(reader)[0])

In [66]:
allword=sorted(allword)
vocab = vocabulary.Vocabulary(allword, size=None)  # size=None means unlimited
print("Vocabulary size: {:,}".format(vocab.size))

Vocabulary size: 5,681


In [7]:
def pad_np_array(example_ids, max_len=50, pad_id=0):

    arr = np.full([len(example_ids), max_len], pad_id, dtype=np.int32)
    ns = np.zeros([len(example_ids)], dtype=np.int32)
    cpy_len = min(len(example_ids), max_len)

    arr = example_ids[:cpy_len]
    ns = cpy_len
    return arr, ns

In [67]:
#Dictionary to collect the tweet detail and the length of each tweet
InputX=defaultdict(dict)
InputN=defaultdict(dict)

In [ ]:
#Set of all the stocks of interest
instock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger','WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']



In [72]:
##Converting the string of data into dictionary


for key,value in d.items():
    Inputx=defaultdict(list)
    Inputn=defaultdict(list)
    if key in instock:
        for value2 in value:
        
            temp=value2.find('), ')
            date=value2[14:temp+3]
            try:
                year=int(date[1:5])
                month=int(date[date.find(',')+2:date.find(',',date.find(',')+1)])
                day=int(date[date.find(')')-2:date.find(')')])
            
                sentence, length = pad_np_array (vocab.words_to_ids(ast.literal_eval(value2[temp+3:-1])))
            
                Inputx[Quarterlookup(DT.datetime(year,month,day))].append(sentence)
                Inputn[Quarterlookup(DT.datetime(year,month,day))].append(length)
            except:
                pass

    InputX[key]=OrderedDict(sorted(Inputx.items()))
    InputN[key]=OrderedDict(sorted(Inputn.items()))

In [19]:
##List of stockname and relevant stock code
l = [("CVSHealth", "CVS"),
("RiteAid", "RAD"),
("Conn's", "CONN"),
("Casey's", "CASY"),
("Kroger", "KR"),
("WeisMarkets", "WMK"),
("HavertyFurniture", "HVT"),
("Kirkland's", "KIRK"),
("Pier1Imports","PIR"),
("BuildersFirstSource","BLDR"),
("HuttigBuildingProducts", "HBP"),
("LumberLiquidators", "LL"),
("TransWorldEntertainment", "TWMC"),
("MarineMax", "HZO"),
("Medifast", "MED"),
("SallyBeauty", "SBH"),
("Sotheby's", "BID"),
("StarGasPartners", "SGU"),
("SuburbanPropanePartners", "SPH"),
("TitanMachinery", "TITN"),
("TractorSupply","TSCO"),
("TravelCentersofAmerica", "TA"),
("Big5SportingGoods", "BGFV"),
("Dick'sSportingGoods", "DKS"),
("HibbettSports", "HIBB"),
("Build-A-BearWorkshop", "BBW")]
     
all_stock = {}
[all_stock.update({k:v}) for k,v in l]
print(all_stock)

{'CVSHealth': 'CVS', 'RiteAid': 'RAD', "Conn's": 'CONN', "Casey's": 'CASY', 'Kroger': 'KR', 'WeisMarkets': 'WMK', 'HavertyFurniture': 'HVT', "Kirkland's": 'KIRK', 'Pier1Imports': 'PIR', 'BuildersFirstSource': 'BLDR', 'HuttigBuildingProducts': 'HBP', 'LumberLiquidators': 'LL', 'TransWorldEntertainment': 'TWMC', 'MarineMax': 'HZO', 'Medifast': 'MED', 'SallyBeauty': 'SBH', "Sotheby's": 'BID', 'StarGasPartners': 'SGU', 'SuburbanPropanePartners': 'SPH', 'TitanMachinery': 'TITN', 'TractorSupply': 'TSCO', 'TravelCentersofAmerica': 'TA', 'Big5SportingGoods': 'BGFV', "Dick'sSportingGoods": 'DKS', 'HibbettSports': 'HIBB', 'Build-A-BearWorkshop': 'BBW'}


In [20]:
##Read the quarterly return detail from edgaronline and convert into library
top="http://datafied.api.edgar-online.com/v2/corefinancials/qtr?primarysymbols="
bottom="&appkey=asc97xrhkyu4959aptu76zxj"
Outputy =defaultdict(dict)
for stockname,stock in all_stock.items():
    check=top+stock+bottom
    with urllib.request.urlopen(check) as url:
        data = json.loads(url.read().decode())
    count1=0
    count2=0
    end_date=[]
    revenue_change=[]
    last_rev=0

    for key, value in data.items():
        
        for key2, value2 in value.items():          
        
            if count1!=0:  
                for i in reversed(value2):
                  
                    for j in i["values"]:
                     
                        if count2!=0:   
                            if j["field"] =="periodenddate":
                                end_date.append(Quarterlookup(DT.datetime.strptime(j['value'], '%m/%d/%Y')))
                       
                            ##Convert revenue into the y_label of 0 for negative growth and 1 for positive growth
                            if j["field"] =="totalrevenue":
                                revenue_cal =round(float(j['value'])/float(last_rev) - 1,3)
                                if revenue_cal>0:
                                    revenue_change.append(1)
                                else:
                                    revenue_change.append(0)
                        else:
                            
                            if j["field"] =="totalrevenue":      
                                last_rev=j['value']
                              
                    count2+=1
            count1+=1
            
    ##Output will be a dictionary with stockname
    Outputy[stockname]["period"]=end_date
    Outputy[stockname]["revenuechange"]=revenue_change

In [21]:
with open('Outputy.txt', 'w') as f:
    print(Outputy, file=f)

In [73]:
train_x = []
train_n = []
train_y = []
test_x = []
test_n = []
test_y = []

In [74]:
quarter=['15Q1','15Q2','15Q3','15Q4','16Q1','16Q2','16Q3','16Q4','17Q1','17Q2','17Q3','17Q4','18Q1','18Q2','18Q3']

for i in instock:
    for j in range(len(quarter)):
        try:
            if quarter[j]=='18Q3':
                test_x.append(InputX [i][quarter[j]])
                test_n.append(InputN [i][quarter[j]])
                length=len(InputN [i][quarter[j]])
                test_y.append([Outputy [i]['revenuechange'][j]]*length)                
               
            else:
                train_x.append(InputX [i][quarter[j]])
                train_n.append(InputN [i][quarter[j]])
                length=len(InputN [i][quarter[j]])           
                train_y.append([Outputy [i]['revenuechange'][j]]*length)
      
        except:
            pass

# NEURAL BAG OF WORDS MODEL #

In [75]:
print(len(train_x[0]))
print(len(train_n[0]))
print(len(train_y[0]))
print(len(train_x[0][0]))
print(train_n[0][0])

print(len(test_x[0]))
print(len(test_n[0]))
print(len(test_y[0]))


23021
23021
23021
34
34
23810
23810
23810


In [ ]:
print(train_x[0][0])

In [40]:
train_x_init = train_x
train_n_init = train_n
train_y_init = train_y

In [42]:
def pad_np_array2(example_ids, max_len=50, pad_id=0):

    arr = np.full([len(example_ids), max_len], pad_id, dtype=np.int32)
    ns = np.zeros([len(example_ids)], dtype=np.int32)
    for i, ids in enumerate(example_ids):
        cpy_len = min(len(ids), max_len)
        arr[i,:cpy_len] = ids[:cpy_len]
        ns[i] = cpy_len
    return arr, ns

In [43]:
print(len(train_x_init[3][4]))
print(train_x_init[3][4])

48
[1946, 3275, 748, 4639, 2, 3399, 178, 2, 199, 1088, 987, 5287, 3522, 2, 1950, 2, 1951, 2, 4925, 2, 5585, 2, 750, 2, 1875, 2, 1876, 2, 907, 2, 5583, 2, 5584, 2, 3057, 2, 749, 2, 3058, 2, 2031, 2, 1029, 2, 1028, 2, 1497, 3663]


In [44]:
train_x = pad_np_array2(train_x_init[0], max_len = 50, pad_id=0)
train_n = np.array(train_n_init[0])
train_y = np.array(train_y_init[0])

test_x = pad_np_array2(train_x_init[1], max_len = 50, pad_id=0)
test_n = np.array(train_n_init[1])
test_y = np.array(train_y_init[1])

dev_x = pad_np_array2(train_x_init[2], max_len = 50, pad_id=0)
dev_n = np.array(train_n_init[2])
dev_y = np.array(train_y_init[2])

In [45]:
print(len(train_x))
print(len(train_n))
print(len(train_y))

print(train_x)
print(train_n)
print(train_y)

print(train_x[0])
print(train_x[1])
# print(train_x[2])

2
23021
23021
(array([[   2, 5095, 2526, ...,    0,    0,    0],
       [5006, 2796, 1790, ..., 3730,    0,    0],
       [1946, 3275,  748, ..., 3663,    0,    0],
       ...,
       [   2, 4696,    2, ...,    0,    0,    0],
       [2678,    2, 5006, ..., 1780,  663, 5678],
       [ 454, 5605, 4241, ...,    0,    0,    0]], dtype=int32), array([34, 48, 48, ..., 39, 50, 25], dtype=int32))
[34 48 48 ... 39 50 25]
[1 1 1 ... 1 1 1]
[[   2 5095 2526 ...    0    0    0]
 [5006 2796 1790 ... 3730    0    0]
 [1946 3275  748 ... 3663    0    0]
 ...
 [   2 4696    2 ...    0    0    0]
 [2678    2 5006 ... 1780  663 5678]
 [ 454 5605 4241 ...    0    0    0]]
[34 48 48 ... 39 50 25]


In [46]:
train_x = np.array(train_x[0])
dev_x = np.array(dev_x[0])
test_x = np.array(test_x[0])

In [47]:
print(len(train_x))
print(len(train_x[0]))
print(len(dev_x))
print(len(dev_x[0]))
print(len(test_x))
print(len(test_x[0]))

23021
50
23693
50
23568
50


In [48]:
print(train_x)
print(train_x[0])

[[   2 5095 2526 ...    0    0    0]
 [5006 2796 1790 ... 3730    0    0]
 [1946 3275  748 ... 3663    0    0]
 ...
 [   2 4696    2 ...    0    0    0]
 [2678    2 5006 ... 1780  663 5678]
 [ 454 5605 4241 ...    0    0    0]]
[   2 5095 2526    2 3148    2  546    2 5094    2 5584    2  685    2
 3058    2 5585    2 4898    2 1267    2 3058 3437 3148 2386    2   64
 2612    2 5321    2 2464 5678    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [49]:
print(len(train_n))
print(len(train_y))

print(type(train_x))
print(type(train_n))
print(type(train_y))

23021
23021
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [56]:
reload(models)

x, ns, y = train_x, train_n, train_y
batch_size = 25

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab.size, embed_dim=50, hidden_dims=[25], num_classes=2,
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.01)
model_fn = models.classifier_model_fn

total_batches = 0
total_examples = 0
total_loss = 0
loss_ema = np.log(2)  # track exponential-moving-average of loss
ema_decay = np.exp(-1/10)  # decay parameter for moving average = np.exp(-1/history_length)
with tf.Graph().as_default(), tf.Session() as sess:
    ##
    # Construct the graph here. No session.run calls - just wiring up Tensors.
    ##
    # Add placeholders so we can feed in data.
    x_ph_  = tf.placeholder(tf.int32, shape=[None, x.shape[1]])  # [batch_size, max_len]
    ns_ph_ = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    y_ph_  = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    
    # Construct the graph using model_fn
    features = {"ids": x_ph_, "ns": ns_ph_}  # note that values are Tensors
    estimator_spec = model_fn(features, labels=y_ph_, mode=tf.estimator.ModeKeys.TRAIN,
                              params=model_params)
    loss_     = estimator_spec.loss
    train_op_ = estimator_spec.train_op
    
    ##
    # Done constructing the graph, now we can make session.run calls.
    ##
    sess.run(tf.global_variables_initializer())
    
    # Run a single epoch
    t0 = time.time()
    for (bx, bns, by) in utils.multi_batch_generator(batch_size, x, ns, y):
        # feed NumPy arrays into the placeholder Tensors
        feed_dict = {x_ph_: bx, ns_ph_: bns, y_ph_: by}
        batch_loss, _ = sess.run([loss_, train_op_], feed_dict=feed_dict)
        
        # Compute some statistics
        total_batches += 1
        total_examples += len(bx)
        total_loss += batch_loss * len(bx)  # re-scale, since batch loss is mean
        # Compute moving average to smooth out noisy per-batch loss
        loss_ema = ema_decay * loss_ema + (1 - ema_decay) * batch_loss
        
        if (total_batches % 25 == 0):
            print("{:5,} examples, moving-average loss {:.2f}".format(total_examples, 
                                                                      loss_ema))    
    print("Completed one epoch in {:s}".format(utils.pretty_timedelta(since=t0)))

  625 examples, moving-average loss 0.54
1,250 examples, moving-average loss 0.35
1,875 examples, moving-average loss 0.27
2,500 examples, moving-average loss 0.23
3,125 examples, moving-average loss 0.20
3,750 examples, moving-average loss 0.17
4,375 examples, moving-average loss 0.15
5,000 examples, moving-average loss 0.13
5,625 examples, moving-average loss 0.12
6,250 examples, moving-average loss 0.11
6,875 examples, moving-average loss 0.10
7,500 examples, moving-average loss 0.09
8,125 examples, moving-average loss 0.08
8,750 examples, moving-average loss 0.07
9,375 examples, moving-average loss 0.07
10,000 examples, moving-average loss 0.06
10,625 examples, moving-average loss 0.06
11,250 examples, moving-average loss 0.06
11,875 examples, moving-average loss 0.05
12,500 examples, moving-average loss 0.05
13,125 examples, moving-average loss 0.05
13,750 examples, moving-average loss 0.05
14,375 examples, moving-average loss 0.05
15,000 examples, moving-average loss 0.04
15,625 

In [57]:
import models; reload(models)

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab.size, embed_dim=50, hidden_dims=[25], num_classes=2,
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.01)

checkpoint_dir = "/tmp/tf_bow_sst_" + datetime.datetime.now().strftime("%Y%m%d-%H%M")
if os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
# Write vocabulary to file, so TensorBoard can label embeddings.
# creates checkpoint_dir/projector_config.pbtxt and checkpoint_dir/metadata.tsv
vocab.write_projector_config(checkpoint_dir, "Encoder/Embedding_Layer/W_embed")

model = tf.estimator.Estimator(model_fn=models.classifier_model_fn, 
                               params=model_params,
                               model_dir=checkpoint_dir)
print("")
print("To view training (once it starts), run:\n")
print("    tensorboard --logdir='{:s}' --port 6006".format(checkpoint_dir))
print("\nThen in your browser, open: http://localhost:6006")

Vocabulary (5,681 words) written to '/tmp/tf_bow_sst_20181202-0353/metadata.tsv'
Projector config written to /tmp/tf_bow_sst_20181202-0353/projector_config.pbtxt
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tf_bow_sst_20181202-0353', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff53e4cc550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

To view training (once it starts), run:

    tensorboard --logdir='/tmp/tf_bow_sst_20181202-0353' --port 6006

Then in your browser, open: http:

In [58]:
# Training params, just used in this cell for the input_fn-s
train_params = dict(batch_size=25, total_epochs=10, eval_every=2)
assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

# Construct and train the model, saving checkpoints to the directory above.
# Input function for training set batches
# Do 'eval_every' epochs at once, followed by evaluating on the dev set.
# NOTE: use patch_numpy_io.numpy_input_fn instead of tf.estimator.inputs.numpy_input_fn
train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": train_x, "ns": train_n}, y=train_y,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

# Input function for dev set batches. As above, but:
# - Don't randomize order
# - Iterate exactly once (one epoch)
dev_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": dev_x, "ns": dev_n}, y=dev_y,
                    batch_size=25, num_epochs=1, shuffle=False
                )

for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    # Train for a few epochs, then evaluate on dev
    model.train(input_fn=train_input_fn)
    eval_metrics = model.evaluate(input_fn=dev_input_fn, name="dev")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tf_bow_sst_20181202-0353/model.ckpt.
INFO:tensorflow:loss = 1.1935925, step = 1
INFO:tensorflow:global_step/sec: 212.56
INFO:tensorflow:loss = 0.29453677, step = 101 (0.473 sec)
INFO:tensorflow:global_step/sec: 240.953
INFO:tensorflow:loss = 0.17239937, step = 201 (0.415 sec)
INFO:tensorflow:global_step/sec: 235.8
INFO:tensorflow:loss = 0.10990701, step = 301 (0.424 sec)
INFO:tensorflow:global_step/sec: 228.663
INFO:tensorflow:loss = 0.077133864, step = 401 (0.438 sec)
INFO:tensorflow:global_step/sec: 227.624
INFO:tensorflow:loss = 0.060306218, step = 501 (0.441 sec)
INFO:tensorflow:global_step/sec: 233.241
INFO:tensorflow:loss = 0.049825072, step = 601 (0.427 sec)
INFO:tensorflow:global_step/sec: 239.6

INFO:tensorflow:global_step/sec: 164.979
INFO:tensorflow:loss = 0.034935027, step = 5485 (0.606 sec)
INFO:tensorflow:Saving checkpoints for 5526 into /tmp/tf_bow_sst_20181202-0353/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0321045.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-02-03:54:42
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181202-0353/model.ckpt-5526
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-12-02-03:54:45
INFO:tensorflow:Saving dict for global step 5526: accuracy = 1.0, cross_entropy_loss = 0.0052720606, global_step = 5526, loss = 0.034114897
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5526: /tmp/tf_bow_sst_20181202-0353/model.ckpt-5526
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Checkpoi

In [59]:

# test_input_fn = None  # replace with an input_fn, similar to dev_input_fn

test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": test_x, "ns": test_n}, y=test_y,
                    batch_size=128, num_epochs=1, shuffle=False
                )

eval_metrics = model.evaluate(input_fn = test_input_fn)  # replace with result of model.evaluate(...)

#### END(YOUR CODE) ####
print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))
eval_metrics

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-02-03:57:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181202-0353/model.ckpt-9210
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-12-02-03:57:10
INFO:tensorflow:Saving dict for global step 9210: accuracy = 1.0, cross_entropy_loss = 0.0052822744, global_step = 9210, loss = 0.05611941
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 9210: /tmp/tf_bow_sst_20181202-0353/model.ckpt-9210
Accuracy on test set: 100.00%


{'accuracy': 1.0,
 'cross_entropy_loss': 0.0052822744,
 'loss': 0.05611941,
 'global_step': 9210}

In [60]:
from sklearn.metrics import accuracy_score
predictions = list(model.predict(test_input_fn))  # list of dicts
y_pred = [p['max'] for p in predictions]
acc = accuracy_score(y_pred, test_y)

print("Accuracy on test set: {:.02%}".format(acc))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181202-0353/model.ckpt-9210
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Accuracy on test set: 100.00%


In [61]:
# predicted labels
print(y_pred[:50])
print(predictions[:2])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[{'proba': array([0.0054353 , 0.99456465], dtype=float32), 'max': 1}, {'proba': array([0.00532511, 0.9946749 ], dtype=float32), 'max': 1}]
